# Exploratory Data Analysis- The Basics
An approach to EDA:  
![image of the data flow showing visualization as an exploratory and iterative process](http://benbestphd.com/images/r4ds_data-science.png)

#### The goal of EDA is to discover patterns in data. This is a fundamental stepping stone towards predictive modelling, or an end goal in itself. 

Tips for good EDA:
- Get to know the context of the data.  
- Question the data: Who collected it? Who is distributing it? Do all of the patterns make sense to what you know about the world? If they don’t, go back and look more closely at your data.

- Use EDA to formulate a question based on the patterns that you see.
- Use EDA to check if a hypothesis is worth a deeper analysis.

- Keep the questions SIMPLE and BRIEF- the goal is to understand and build complexity further on.
- Its an iterative process-- its okay to repeat things so long as you learn from previous output.

In [1]:
# importing the libraries for data processing
import numpy as np 
import pandas as pd 


### 1. Tidying our charts data

Read the csv file, check for missing, duplicated and unexpected values, and filtering if needed

In [2]:
# read the charts dataset
charts_df = pd.read_csv('data/spotify_daily_charts.csv')
charts_df.head()

,date,position,track_id,track_name,artist,streams
0,2017-01-01,1,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,Bruno Mars,185236
1,2017-01-01,2,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,James Arthur,180552
2,2017-01-01,3,7BKLCZ1jbUBVqRi2FVlTVw,Closer,The Chainsmokers,158720
3,2017-01-01,4,2rizacJSyD9S1IQUxUxnsK,All We Know,The Chainsmokers,130874
4,2017-01-01,5,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,Maroon 5,129656


### Data Checks
It is prudent to do the following on a DataFrame before any analysis is made
1. Check shape
2. Check data types of columns
3. Check null values in columns
4. Check rows with null values
5. Check for duplicates

In [3]:
#Check the shape of the dataframe
charts_df.shape 

(319600, 6)

In [4]:
#list comprehension to check elements of a list not in another list
complete_dates = pd.date_range(start='2017-01-01', end='2021-05-20', freq='D').strftime('%Y-%m-%d')
#complete_dates = pd.date_range(start='2017-01-01', end='2021-05-20', freq='D')#.strftime('%Y-%m-%d')
dataset_dates = pd.unique(charts_df['date'])

[p for p in complete_dates if p not in dataset_dates]

['2017-05-30', '2017-05-31', '2017-06-02']

In [17]:
dataset_dates

array(['2017-01-01', '2017-01-02', '2017-01-03', ..., '2021-05-18',
       '2021-05-19', '2021-05-20'], dtype=object)

In [18]:
charts_df['date'].unique()

array(['2017-01-01T00:00:00.000000000', '2017-01-02T00:00:00.000000000',
       '2017-01-03T00:00:00.000000000', ...,
       '2021-05-18T00:00:00.000000000', '2021-05-19T00:00:00.000000000',
       '2021-05-20T00:00:00.000000000'], dtype='datetime64[ns]')

In [5]:
200*len(charts_df['date'].unique())

319600

In [6]:
#Check data types of the columns
charts_df.dtypes

date          object
position       int64
track_id      object
track_name    object
artist        object
streams        int64
dtype: object

In [7]:
#Check null values in the columns
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319600 entries, 0 to 319599
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        319600 non-null  object
 1   position    319600 non-null  int64 
 2   track_id    319600 non-null  object
 3   track_name  319590 non-null  object
 4   artist      319590 non-null  object
 5   streams     319600 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 14.6+ MB


In [8]:
charts_df[charts_df['artist'].isnull()]

,date,position,track_id,track_name,artist,streams
39406,2017-07-20,7,3RXkboS74UYzN14xTqzPyY,NaN,NaN,121541
39423,2017-07-20,24,3eFJqPe8VUYrABbFjSauuj,NaN,NaN,85374
39426,2017-07-20,27,4JAyIDXOqNM6qHuZML01uX,NaN,NaN,81223
39455,2017-07-20,56,3bVbQvGVIe4n24AzyXovXh,NaN,NaN,46754
39606,2017-07-21,7,3RXkboS74UYzN14xTqzPyY,NaN,NaN,120186
39621,2017-07-21,22,4JAyIDXOqNM6qHuZML01uX,NaN,NaN,92580
39628,2017-07-21,29,3eFJqPe8VUYrABbFjSauuj,NaN,NaN,83205
39690,2017-07-21,91,3bVbQvGVIe4n24AzyXovXh,NaN,NaN,35049
39902,2017-07-22,103,6Br5mChPdgQNmLF0G0gjPH,NaN,NaN,29422
40098,2017-07-23,99,6Br5mChPdgQNmLF0G0gjPH,NaN,NaN,28054


In [40]:
charts_df.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
319595    False
319596    False
319597    False
319598    False
319599    False
Length: 319600, dtype: bool

In [9]:
#Check for duplicates
sum(charts_df.duplicated())

0

In [10]:
#check if unique values are expected
charts_df['position'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [11]:
len(charts_df['artist'].unique())

801

In [12]:
len(charts_df['track_name'].unique())

2742

In [13]:
len(charts_df['track_id'].unique())

3495

> Q: Why do we have N track ids but only M track names?

##### Convert date to datetime index
Pandas has a very useful method `pd.to_datetime` that smartly recognizes date and time columns and allows for easier time series techniques

In [14]:
#transform date column into a datetime column
charts_df['date'] = pd.to_datetime(charts_df['date'])
charts_df.head()

,date,position,track_id,track_name,artist,streams
0,2017-01-01,1,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,Bruno Mars,185236
1,2017-01-01,2,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,James Arthur,180552
2,2017-01-01,3,7BKLCZ1jbUBVqRi2FVlTVw,Closer,The Chainsmokers,158720
3,2017-01-01,4,2rizacJSyD9S1IQUxUxnsK,All We Know,The Chainsmokers,130874
4,2017-01-01,5,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,Maroon 5,129656


In [15]:
#extract month 
charts_df['month']=charts_df['date'].dt.month
charts_df.head()

,date,position,track_id,track_name,artist,streams,month
0,2017-01-01,1,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,Bruno Mars,185236,1
1,2017-01-01,2,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,James Arthur,180552,1
2,2017-01-01,3,7BKLCZ1jbUBVqRi2FVlTVw,Closer,The Chainsmokers,158720,1
3,2017-01-01,4,2rizacJSyD9S1IQUxUxnsK,All We Know,The Chainsmokers,130874,1
4,2017-01-01,5,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,Maroon 5,129656,1


In [16]:
#extract year
charts_df['year']=charts_df['date'].dt.year
# get day and day of week
charts_df['day']=charts_df['date'].dt.day
charts_df['day_of_week']=charts_df['date'].dt.dayofweek # The day of the week with Monday=0, Sunday=6.
charts_df.head()

,date,position,track_id,track_name,artist,streams,month,year,day,day_of_week
0,2017-01-01,1,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,Bruno Mars,185236,1,2017,1,6
1,2017-01-01,2,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,James Arthur,180552,1,2017,1,6
2,2017-01-01,3,7BKLCZ1jbUBVqRi2FVlTVw,Closer,The Chainsmokers,158720,1,2017,1,6
3,2017-01-01,4,2rizacJSyD9S1IQUxUxnsK,All We Know,The Chainsmokers,130874,1,2017,1,6
4,2017-01-01,5,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,Maroon 5,129656,1,2017,1,6


### 2. Examining the charts data
Reshape and aggregate the DataFrame to answer basic data questions 

In [19]:
#Lets create tallies of each column using the `value_counts` method
charts_df['artist'].value_counts()[:20]

Ed Sheeran          9657
LANY                8629
Ben&Ben             8066
Moira Dela Torre    6450
December Avenue     5946
Lauv                5457
Ariana Grande       5248
Taylor Swift        5144
Maroon 5            4609
BLACKPINK           4563
The Chainsmokers    4286
Post Malone         4276
Bruno Mars          4185
Silent Sanctuary    3935
BTS                 3836
Hale                3283
Sam Smith           3281
Jason Mraz          3212
Dua Lipa            3155
South Border        3002
Name: artist, dtype: int64

In [20]:
charts_df['track_name'].value_counts()

Happier                                            1628
Your Song                                          1600
Say You Won't Let Go                               1598
Tadhana                                            1596
Tell Me Where It Hurts                             1595
                                                   ... 
Mob Ties                                              1
Midnight Memories                                     1
Dati - Cover Version                                  1
Taki Taki (feat. Selena Gomez, Ozuna & Cardi B)       1
DON’T CALL ME AGAIN                                   1
Name: track_name, Length: 2741, dtype: int64

In [21]:
#filtering columns
charts_df[charts_df['track_name']=='Happier']

,date,position,track_id,track_name,artist,streams,month,year,day,day_of_week
12230,2017-03-03,31,2RttW7RAu5nOAfq6YFvApB,Happier,Ed Sheeran,84942,3,2017,3,4
12425,2017-03-04,26,2RttW7RAu5nOAfq6YFvApB,Happier,Ed Sheeran,97382,3,2017,4,5
12619,2017-03-05,20,2RttW7RAu5nOAfq6YFvApB,Happier,Ed Sheeran,108776,3,2017,5,6
12815,2017-03-06,16,2RttW7RAu5nOAfq6YFvApB,Happier,Ed Sheeran,116598,3,2017,6,0
13014,2017-03-07,15,2RttW7RAu5nOAfq6YFvApB,Happier,Ed Sheeran,121400,3,2017,7,1
...,...,...,...,...,...,...,...,...,...,...
318797,2021-05-16,198,2RttW7RAu5nOAfq6YFvApB,Happier,Ed Sheeran,24137,5,2021,16,6
318983,2021-05-17,184,2RttW7RAu5nOAfq6YFvApB,Happier,Ed Sheeran,25737,5,2021,17,0
319185,2021-05-18,186,2RttW7RAu5nOAfq6YFvApB,Happier,Ed Sheeran,27716,5,2021,18,1
319388,2021-05-19,189,2RttW7RAu5nOAfq6YFvApB,Happier,Ed Sheeran,27473,5,2021,19,2


> Q1. From top 50 most streamed, get top 20 most frequently occuring artists

In [22]:
charts_df[charts_df['position']<=50]['artist'].value_counts()[:20]

Ben&Ben                4404
Moira Dela Torre       2502
December Avenue        2348
Ed Sheeran             2020
LANY                   2015
Ariana Grande          1902
Lauv                   1884
Dua Lipa               1436
Maroon 5               1375
The Chainsmokers       1312
Taylor Swift           1293
Post Malone            1262
BTS                    1102
This Band              1085
I Belong to the Zoo    1067
BLACKPINK              1065
Justin Bieber          1037
Khalid                  994
Shawn Mendes            915
Jonas Blue              910
Name: artist, dtype: int64

> Q2. From top 50 list this year, get top 20 most frequently occuring artists

In [23]:
charts_df[(charts_df['position']>=50)&(charts_df['year']==2020)]['artist'].value_counts()[:20]

Taylor Swift        1687
Lauv                1608
December Avenue     1583
Ben&Ben             1453
Moira Dela Torre    1249
LANY                1246
Ed Sheeran          1233
BTS                 1142
TWICE               1014
BLACKPINK            913
Harry Styles         856
Sam Smith            773
This Band            739
Hale                 731
Jason Mraz           700
South Border         689
Matthaios            672
Post Malone          656
Shawn Mendes         611
Lady Gaga            606
Name: artist, dtype: int64

> Q3. On what positions did Taylor Swift land on the chart for 2019? What were her songs that landed first on the chart?

In [24]:
np.sort(charts_df[(charts_df['artist']=='Taylor Swift')&(charts_df['year']==2019)]['position'].unique())

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 150, 152, 153, 154, 155, 156, 157, 158,
       159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
       172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 18

In [25]:
charts_df[(charts_df['artist']=='Taylor Swift')&\
                    (charts_df['year']==2019)&\
                    (charts_df['position']==1)]['track_name'].unique()

array(['ME! (feat. Brendon Urie of Panic! At The Disco)', 'Lover'],
      dtype=object)

### 3. Quick stats and Aggregating the charts dataset


**Quick stats**


Basic stats can be computed using the `describe` method

In [26]:
charts_df['streams'].describe()

count    319600.000000
mean      50986.204662
std       41451.244331
min       11004.000000
25%       26507.750000
50%       34284.000000
75%       59159.250000
max      670517.000000
Name: streams, dtype: float64

**Aggregation**

The pandas `groupby` method functions in the same way as pivot_table in excel

The syntax for a single index column and single agg column:
```python
df.groupby('index_col')['agg_col'].aggfunc()
```

A good analogy for  pandas `groupby` is making cocktails at a party: the glasses is the items in `index_col`, the beverage is the `agg_col`, and how the beverage is poured into the glasses is `aggfunc`.

<img src="groupby.png" align="left" alt="Drawing" style="width: 300px;"/>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>


For multiple indices and multiple aggregations:
```python
df.groupby(['index_col','index_col2']).agg('agg_col1':aggfunc1, 'agg_col2':aggfunc2)
```

Q: How many total streams did charting songs in Spotify earn per year?

In [27]:
charts_df.groupby('year')['streams'].sum()   #inputting a column name string in agg_column outputs a Series

year
2017    3242144459
2018    3467089600
2019    4081571771
2020    4000302402
2021    1504082778
Name: streams, dtype: int64

In [28]:
charts_df.groupby('year')[['streams']].sum()   #inputting a list in agg_column outputs a DataFrame

,streams
year,
2017,3242144459
2018,3467089600
2019,4081571771
2020,4000302402
2021,1504082778


> Q: How many streams did each of the 200 positions contribute to the annual streams of spotify?

In [29]:
charts_df.groupby(['year','position'])[['streams']].sum()   #inputting a list in agg_column outputs a DataFrame

streams
year position          
2017 1         91459424
     2         76542555
     3         68385212
     4         60809129
     5         57562776
...                 ...
2021 196        3728988
     197        3719677
     198        3710615
     199        3700643
     200        3691132

[1000 rows x 1 columns]

> Q: What visualization would best suit the output of the cell above?

### 4. Combining two datasets

- What insights could we get from merging the charts and tracks datasets?

In [30]:
# read the tracks dataset
tracks_df = pd.read_csv('data/spotify_daily_charts_tracks.csv')
tracks_df.head()

,track_id,track_name,artist_id,artist_name,album_id,duration,release_date,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,4PgleR09JVnm3zY1fW3XBA,261240,2016-11-17,76,0.578,0.574,2.0,-6.209,1.0,0.0454,0.196,0.00000,0.0830,0.301,174.152
1,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,4IWBUUAFIplrNtaOHcJPRM,James Arthur,7oiJYvEJHsmYtrgviAVIBD,211466,2016-10-28,85,0.358,0.557,10.0,-7.398,1.0,0.0590,0.695,0.00000,0.0902,0.494,85.043
2,7BKLCZ1jbUBVqRi2FVlTVw,Closer,69GGBxA162lTqCwzJG5jLp,The Chainsmokers,0rSLgV8p5FzfnqlEk4GzxE,244960,2016-07-29,85,0.748,0.524,8.0,-5.599,1.0,0.0338,0.414,0.00000,0.1110,0.661,95.010
3,2rizacJSyD9S1IQUxUxnsK,All We Know,69GGBxA162lTqCwzJG5jLp,The Chainsmokers,0xmaV6EtJ4M3ebZUPRnhyb,194080,2016-09-29,69,0.662,0.586,0.0,-8.821,1.0,0.0307,0.097,0.00272,0.1150,0.296,90.000
4,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,04gDigrS5kc9YWfZHwBETP,Maroon 5,0fvTn3WXF39kQs9i3bnNpP,214480,2016-10-11,0,0.783,0.623,7.0,-6.126,1.0,0.0800,0.338,0.00000,0.0975,0.447,100.048


**Combining dataframes*

The pandas `merge` method combines DataFrames/Series in the same way common database languages (eg. SQL) perform table joins.

The most basic syntax for a single index column and single agg column:
```python
df1.merge(df2, on='key_column', how=<'join_type'>)
```
where <'join_type'> = ['left','right','inner','outer']

Here is a diagram that illustrates what each of the join type produces
<br>
<br>
<img src="merge.png" align="left" alt="Drawing" style="width: 500px;"/>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
For multiple key columns:
```python
df1.merge(df2, on=['key_column1','key_column2'], how=<'join_type'>)
```

In [31]:
#merge charts dataframe with tracks dataframe
#follow charts_df's rows

df = charts_df.merge(tracks_df, on='track_id', how='left')
df.head()

,date,position,track_id,track_name_x,artist,streams,month,year,day,day_of_week,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2017-01-01,1,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,Bruno Mars,185236,1,2017,1,6,...,0.574,2.0,-6.209,1.0,0.0454,0.196,0.00000,0.0830,0.301,174.152
1,2017-01-01,2,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,James Arthur,180552,1,2017,1,6,...,0.557,10.0,-7.398,1.0,0.0590,0.695,0.00000,0.0902,0.494,85.043
2,2017-01-01,3,7BKLCZ1jbUBVqRi2FVlTVw,Closer,The Chainsmokers,158720,1,2017,1,6,...,0.524,8.0,-5.599,1.0,0.0338,0.414,0.00000,0.1110,0.661,95.010
3,2017-01-01,4,2rizacJSyD9S1IQUxUxnsK,All We Know,The Chainsmokers,130874,1,2017,1,6,...,0.586,0.0,-8.821,1.0,0.0307,0.097,0.00272,0.1150,0.296,90.000
4,2017-01-01,5,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,Maroon 5,129656,1,2017,1,6,...,0.623,7.0,-6.126,1.0,0.0800,0.338,0.00000,0.0975,0.447,100.048


In [32]:
#Always check number of rows when performing merges
charts_df.shape, tracks_df.shape, df.shape

((319600, 10), (3495, 19), (319600, 28))

In [33]:
df.columns

Index(['date', 'position', 'track_id', 'track_name_x', 'artist', 'streams',
       'month', 'year', 'day', 'day_of_week', 'track_name_y', 'artist_id',
       'artist_name', 'album_id', 'duration', 'release_date', 'popularity',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

In [34]:
#drop duplicated track_name column
df = df.drop(columns='track_name_y')
#rename track_name x
df = df.rename(columns={'track_name_x':'track_name'})
df.head()

,date,position,track_id,track_name,artist,streams,month,year,day,day_of_week,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2017-01-01,1,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,Bruno Mars,185236,1,2017,1,6,...,0.574,2.0,-6.209,1.0,0.0454,0.196,0.00000,0.0830,0.301,174.152
1,2017-01-01,2,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,James Arthur,180552,1,2017,1,6,...,0.557,10.0,-7.398,1.0,0.0590,0.695,0.00000,0.0902,0.494,85.043
2,2017-01-01,3,7BKLCZ1jbUBVqRi2FVlTVw,Closer,The Chainsmokers,158720,1,2017,1,6,...,0.524,8.0,-5.599,1.0,0.0338,0.414,0.00000,0.1110,0.661,95.010
3,2017-01-01,4,2rizacJSyD9S1IQUxUxnsK,All We Know,The Chainsmokers,130874,1,2017,1,6,...,0.586,0.0,-8.821,1.0,0.0307,0.097,0.00272,0.1150,0.296,90.000
4,2017-01-01,5,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,Maroon 5,129656,1,2017,1,6,...,0.623,7.0,-6.126,1.0,0.0800,0.338,0.00000,0.0975,0.447,100.048


In [35]:
#check if expected columns are present
df.columns

Index(['date', 'position', 'track_id', 'track_name', 'artist', 'streams',
       'month', 'year', 'day', 'day_of_week', 'artist_id', 'artist_name',
       'album_id', 'duration', 'release_date', 'popularity', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

In [41]:
df.columns.sort_values()

Index(['acousticness', 'album_id', 'artist', 'artist_id', 'artist_name',
       'danceability', 'date', 'day', 'day_of_week', 'duration', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'month',
       'popularity', 'position', 'release_date', 'speechiness', 'streams',
       'tempo', 'track_id', 'track_name', 'valence', 'year'],
      dtype='object')

## Q&A

Q1: What are the top 10 songs in terms of total streams from 2017 to 2020?

In [36]:
# groupby tracks and sum streams, sort and get first 10 rows 
df.groupby(['track_id','track_name'])['streams'].sum().sort_values(ascending=False)[:10]

track_id                track_name            
3WUEs51GpcvlgU7lehLgLh  Kathang Isip              137574061
2BgD4nRyx9EZ5o8YEnjRSV  Kung 'Di Rin Lang Ikaw    124246178
1yDiru08Q6omDOGkZMPnei  Maybe The Night           117216219
1X4l4i472kW5ofFP8Xo0x0  Sana                      116625610
5uCax9HTNlzGybIStD3vDh  Say You Won't Let Go      115333908
00mBzIWv5gHOYxwuEJXjOG  Sa Ngalan Ng Pag-Ibig      96925070
4u8RkgV6P4TLi89SmlUtv8  Mundo                      94473164
5l9g7py8RCblcvbZgGQgSd  Pagtingin                  84711677
0tgVpDi06FyKpA1z0VMD4v  Perfect                    82627480
6NFKf8vBApSvtzkapYmnVR  Tadhana                    79637034
Name: streams, dtype: int64

Q2: Whats the mean tempo of the top 10 most streamed songs?

In [37]:
top10songs = df.groupby(['track_id','track_name'])['streams'].sum()\
            .sort_values(ascending=False)[:10]\
            .reset_index()['track_id'].values
top10songs

array(['3WUEs51GpcvlgU7lehLgLh', '2BgD4nRyx9EZ5o8YEnjRSV',
       '1yDiru08Q6omDOGkZMPnei', '1X4l4i472kW5ofFP8Xo0x0',
       '5uCax9HTNlzGybIStD3vDh', '00mBzIWv5gHOYxwuEJXjOG',
       '4u8RkgV6P4TLi89SmlUtv8', '5l9g7py8RCblcvbZgGQgSd',
       '0tgVpDi06FyKpA1z0VMD4v', '6NFKf8vBApSvtzkapYmnVR'], dtype=object)

In [38]:
#isin selects elements in list
df[df['track_id'].isin(top10songs)]['tempo'].mean() #in bpm

119.00216841506057

Q2a. Follow-up: How does this compare with the mean tempo of the rest of the songs?

In [39]:
#use ~ to negate
df[~df['track_id'].isin(top10songs)]['tempo'].mean() #in bpm

116.94294807967712

**Self Check**

Q: Which song had the most days within top 5 of the charts for 2020?

In [73]:
df.columns.sort_values()

Index(['acousticness', 'album_id', 'artist', 'artist_id', 'artist_name',
       'danceability', 'date', 'day', 'day_of_week', 'duration', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'month',
       'popularity', 'position', 'release_date', 'speechiness', 'streams',
       'tempo', 'track_id', 'track_name', 'valence', 'year'],
      dtype='object')

In [76]:
charts_df[ ((charts_df['year']==2020) & (charts_df['position']<=5))]['track_name'].value_counts().head()

Dynamite                    130
Beautiful Scars             126
Imahe                       116
Intentions (feat. Quavo)    109
Don't Start Now              87
Name: track_name, dtype: int64

Q: Which artist had the most days within top 10 of the charts for 2020?

In [80]:
charts_df[ ((charts_df['year']==2020) & (charts_df['position']<=10))]['artist'].value_counts().head(n=10)

BLACKPINK        333
Justin Bieber    325
Ben&Ben          251
Ariana Grande    240
Maximillian      234
BTS              202
Magnus Haven     175
Emman            172
Dua Lipa         133
Juan Caoile      110
Name: artist, dtype: int64

In [81]:
# for 2021
charts_df[ ((charts_df['year']==2021) & (charts_df['position']<=10))]['artist'].value_counts().head(n=10)

Olivia Rodrigo      158
Pink Sweat$         134
BTS                 133
Justin Bieber       111
Ariana Grande        99
Giveon               97
Skusta Clee          91
Zack Tabudlo         81
Bruno Mars           77
Moira Dela Torre     75
Name: artist, dtype: int64

 Q: What are the top 5 “saddest” charting songs for 2020? 

In [82]:
charts_df.columns.sort_values()

Index(['artist', 'date', 'day', 'day_of_week', 'month', 'position', 'streams',
       'track_id', 'track_name', 'year'],
      dtype='object')

In [84]:
df[ (df['year']==2020) ]\
   .drop_duplicates(['track_id','track_name'])\
   .sort_values('valence')[:5]\
    [['track_name','artist','valence']]

,track_name,artist,valence
272088,Ew,Joji,0.0376
273493,sad,LANY,0.0404
259535,Delicate,Taylor Swift,0.0499
272103,Your Man,Joji,0.0503
227263,No Time To Die,Billie Eilish,0.0517


### Plain tables as output?
1. Tables are simple fast answers to simple fast questions
2. Tables are very useful for troubleshooting. The numbers often reveal if there was something wrong with the data source/processing
3. In most office setups, analtyics output are often offtaked by another team (e.g. market segments group -> finance for sales projections). As it could be readily plugged into their computations, they usually prefer tables instead of deployed products.